# EMS Vehicle Staging Staging Location Optimization on I-76 in Pennsylvania

This notebook performs an MILP optimization analysis to determine optimal staging locations for EMS vehicles along the I-76 corridor in Pennsylvania. It utilizes crash data per segment from the predictions and potential intersection locations to solve a facility location problem.

## Data Required:

1.  **`aggregated_data.csv`**: Contains information about crash hotspots along I-76, including location (latitude and longitude), crash numbers ('CRN'), and severity counts ('is_severe').
2.  **`i76_exits.csv`**: Contains a list of potential ambulance staging locations (intersections/exits) along I-76, including their latitude, longitude, and OpenStreetMap (OSM) IDs.

Please upload these files to your Google Colab environment or ensure the file paths in the notebook are updated to point to their correct locations (e.g., if stored in Google Drive).

## What the Notebook Does:

1.  **Loads Data**: Loads the crash hotspot and intersection data into pandas DataFrames.
2.  **Calculates Distances/Travel Times**: Computes the distance or estimated travel time between each potential intersection location and each crash hotspot.
3.  **Formulates and Solves Optimization Models**:
    *   **Initial Model**: Minimizes the maximum distance from a chosen location to any hotspot.
    *   **Weighted Models**: Minimizes the weighted sum of distances/travel times, using crash numbers ('CRN') or severity ('is_severe') as weights.
    *   **Multiple Facilities Model**: Extends the weighted model to find optimal locations for a varying number of ambulances (2 to 15) - in draft.
4.  **Visualizes Results**: Generates maps to visualize the distribution of hotspots, potential locations, and the optimal locations found by the different models.
5.  **Tabulates Results**: Presents the results for the multiple ambulance model in a table showing the number of ambulances and the corresponding maximum response time - in draft.

## How to Use:

*   Load data called `predictions_Q1_2025.csv` and `i-76_exits.csv`.
*   Ensure the required data files (`predictions_Q1_2025.csv` and `i76_exits.csv`) are accessible in the Colab environment/ local environment - navigate to the folder tab and add these to Colab, or pull them from the zip folders.
*   Explore the different optimization models and visualizations to understand the impact of different criteria on the optimal staging locations.

# Optimization Model Formulations

## 1. Initial Model (Minimizing Maximum Distance)

This model aims to find a single ambulance staging location that minimizes the maximum travel distance to any crash hotspot - this is to confirm that the mechanics of the model is generalizable.

**Sets:**

*   $I$: Set of potential EMS staging locations (intersections).
*   $J$: Set of crash hotspots.

**Parameters:**

*   $d_{ij}$: Distance between location $i \in I$ and hotspot $j \in J$.

**Decision Variables:**

*   $y_i \in \{0, 1\}$: Binary variable; 1 if location $i$ is chosen as a staging location, 0 otherwise.
*   $D_{\text{max}} \ge 0$: Continuous variable representing the maximum distance from a chosen location to any hotspot.

**Objective Function:**

$$ \min D_{\text{max}} $$

**Constraints:**

$$
\begin{align*}
\sum_{i \in I} y_i &= 1 && \text{(Choose exactly one location)} \\
d_{ij} \cdot y_i &\le D_{\text{max}} & \forall i \in I, j \in J && \text{(Maximum distance constraint)}
\end{align*}
$$
Note: The second constraint can be simplified in implementation as $\sum_{i \in I} d_{ij} \cdot y_i \le D_{\text{max}}$ for each $j \in J$, since only one $y_i$ will be 1.

## 2. Weighted Optimization Model (Weighted by CRN)

This model aims to find a single EMS staging location that minimizes the weighted sum of distances to all crash hotspots, where the weights are based on the number of crashes (CRN) at each hotspot.

**Sets:**

*   $I$: Set of potential ambulance staging locations (intersections).
*   $J$: Set of crash hotspots.

**Parameters:**

*   $d_{ij}$: Distance between location $i \in I$ and hotspot $j \in J$.
*   $w_j$: Weight for hotspot $j \in J$, based on its CRN value (e.g., $w_j = \frac{\text{CRN}_j}{\sum_{k \in J} \text{CRN}_k}$).

**Decision Variables:**

*   $y_i \in \{0, 1\}$: Binary variable; 1 if location $i$ is chosen as a staging location, 0 otherwise.

**Objective Function:**

$$ \min \sum_{i \in I} \sum_{j \in J} d_{ij} \cdot y_i \cdot w_j $$

**Constraints:**

$$ \sum_{i \in I} y_i = 1 $$
Choose exactly one location

## 3. Weighted Optimization Model (Weighted by Severity)

This model is similar to the CRN-weighted model but uses the severity of crashes ('is\_severe') as the weight for each hotspot.

**Sets:**

*   $I$: Set of potential ambulance staging locations (intersections).
*   $J$: Set of crash hotspots.

**Parameters:**

*   $d_{ij}$: Distance between location $i \in I$ and hotspot $j \in J$.
*   $s_j$: Weight for hotspot $j \in J$, based on its severity count (e.g., $s_j = \frac{\text{is\_severe}_j}{\sum_{k \in J} \text{is\_severe}_k}$).
\end{itemize}

**Decision Variables:**

*   $y_i \in \{0, 1\}$: Binary variable; 1 if location $i$ is chosen as a staging location, 0 otherwise.

**Objective Function:**

$$ \min \sum_{i \in I} \sum_{j \in J} d_{ij} \cdot y_i \cdot s_j $$

**Constraints:**

$$ \sum_{i \in I} y_i = 1 $$
Choose exactly one location

**Severity Weighted Optimization Model Summary (One EMS)**

*   **Parameters:** $d_{ij}$ (distance from location $i$ to hotspot $j$), $s_j$ (severity weight of hotspot $j$).
*   **Decision Variables:** $y_i$ (binary for placing ambulance at $i$).
*   **Objective:** Minimize $\sum_{i \in I} \sum_{j \in J} d_{ij} \cdot y_i \cdot s_j$ (Minimize severity-weighted total distance).
*   **Constraints:** 1. Choose exactly one location: $\sum_{i \in I} y_i = 1$.

**Accessing relevant libraries, loading data - post preperation**

In [13]:
#installing relevant libraries and using data from drive
#using pulp for MIP
#installing openstreet maps - OSMnx

!pip install pulp osmnx geopandas folium geopy

import pandas as pd
import geopy.distance
import folium
from pulp import *
import osmnx as ox

crash_hotspots = pd.read_csv('../data/predictions_Q1_2025.csv')
intersections = pd.read_csv('../data/i76_exits.csv')
print(crash_hotspots, "\n")
print(intersections.head(10))

   28milename  14milelat  14milelon  CRN  is_severe
0     mile112  40.107882 -79.242567   32          1
1     mile140  39.964591 -78.809443   17          1
2     mile168  40.004740 -78.382198   21          0
3     mile196  40.067397 -77.926592   22          1
4     mile224  40.187713 -77.463394   18          1
5     mile252  40.199970 -76.948185   57          2
6      mile28  40.697655 -80.298905   24          1
7     mile280  40.228711 -76.450034   26          2
8     mile308  40.174169 -75.961205   55          2
9     mile336  40.075964 -75.491609  168          3
10     mile56  40.588292 -79.883370   60          2
11     mile84  40.264796 -79.661088   34          1 

  Exit_Number              Name   Latitude  Longitude      OSM_ID
0          10        New Castle  40.840480 -80.384527   200659479
1          10               NaN  40.834572 -80.372258   310943605
2          13     Beaver Valley  40.813677 -80.324237   200698653
3          13     Beaver Valley  40.813054 -80.322712   31

In [14]:
# Create a dictionary to hold the distance between each intersection and hotspot
distance_matrix = {}

# Loop through each intersection
for int_idx, intersection in intersections.iterrows():
    int_id = intersection['OSM_ID']
    int_coords = (intersection['Latitude'], intersection['Longitude'])
    distance_matrix[int_id] = {}

    # Loop through each hotspot to calculate the distance
    for hot_idx, hotspot in crash_hotspots.iterrows():
        hot_id = hotspot['28milename']
        hot_coords = (hotspot['14milelat'], hotspot['14milelon'])

        # Calculate and store the distance
        dist = geopy.distance.great_circle(int_coords, hot_coords).miles
        distance_matrix[int_id][hot_id] = dist

# Print the matrix in a readable format
pd.DataFrame(distance_matrix).round(2)

,200659479,310943605,200698653,311343253,311436261,755496289,713280848,713282313,1463828484,1463828485,...,148576041,148576073,200916833,201810722,444353744,746197933,851374264,1463828588,1838815415,5956886759
mile112,78.51,77.76,74.91,74.82,59.79,60.33,50.65,50.64,42.70,43.11,...,61.40,61.20,39.75,40.27,12.63,74.82,200.79,41.69,131.95,183.27
mile140,102.62,101.86,98.97,98.89,83.89,84.40,74.59,74.58,66.58,66.94,...,38.97,38.80,17.21,17.58,12.92,98.88,178.27,65.66,110.32,160.91
mile168,120.11,119.35,116.46,116.36,101.68,102.14,92.32,92.32,84.45,84.74,...,16.24,16.08,6.40,5.84,35.28,116.36,155.52,83.70,87.56,138.14
mile196,139.82,139.08,136.21,136.12,121.92,122.33,112.70,112.69,105.10,105.33,...,8.30,8.48,30.26,29.82,59.54,136.12,131.28,104.48,63.07,113.83
mile224,159.92,159.20,156.39,156.31,142.76,143.11,133.79,133.79,126.56,126.73,...,33.95,34.11,55.65,55.25,84.78,156.31,106.94,126.08,37.83,89.33
mile252,185.82,185.11,182.34,182.25,169.03,169.36,160.19,160.19,153.12,153.27,...,60.75,60.93,82.64,82.22,111.86,182.25,79.88,152.68,10.63,62.24
mile28,10.84,10.21,8.13,8.07,10.46,10.15,19.53,19.53,27.26,26.99,...,124.74,124.53,105.36,105.94,81.55,8.06,258.98,28.01,189.73,241.34
mile280,210.87,210.16,207.44,207.35,194.43,194.74,185.71,185.71,178.79,178.92,...,87.06,87.23,108.97,108.55,138.20,207.35,54.10,178.39,15.73,36.46
mile308,236.86,236.16,233.44,233.36,220.48,220.79,211.78,211.78,204.86,204.99,...,112.40,112.58,134.36,133.92,163.64,233.36,28.07,204.46,41.56,10.56
mile336,262.56,261.86,259.14,259.05,246.12,246.44,237.39,237.38,230.42,230.55,...,137.03,137.21,158.97,158.51,188.22,259.05,2.58,230.00,66.97,15.24


**Formulation of Problem**

In [16]:
# 1 - Initializing the Model
model = LpProblem("EMS_Location", LpMinimize) #we want to minimize Z

# 2 - Decision Variables are set up
intersection_ids = intersections['OSM_ID'].tolist()
hotspot_ids = crash_hotspots['28milename'].tolist()

# 'y_i' - A binary variable for each intersection. 1 if we choose it, 0 if not.
location_vars = LpVariable.dicts("y", intersection_ids, cat='Binary')

# 'D_max' - A continuous variable representing the max distance we want to minimize.
max_dist_var = LpVariable("D_max", lowBound=0, cat='Continuous')


# 3 - Optimization function set up
model += max_dist_var, "Minimize_Maximum_Distance" #minimize Dmax


# 4 - Constraint set up
# Constraint 1: We must choose exactly ONE intersection.
model += lpSum([location_vars[i] for i in intersection_ids]) == 1, "Choose_Exactly_One_Intersection"
# Constraint 2: For each hotspot, the distance from the chosen location must be less than or equal to D_max.
for j in hotspot_ids:
    model += lpSum([distance_matrix[i][j] * location_vars[i] for i in intersection_ids]) <= max_dist_var, f"Max_Distance_to_Hotspot_{j}"

model #print the model

EMS_Location:
MINIMIZE
1*D_max + 0
SUBJECT TO
Choose_Exactly_One_Intersection: y_1046413828 + y_106974535 + y_107295090
 + y_108367367 + y_109742507 + y_109748357 + y_109748703 + y_109748955
 + y_109761712 + y_109773755 + y_109774579 + y_109781521 + y_109786935
 + y_109788517 + y_109915821 + y_109916149 + y_109916625 + y_109917904
 + y_109918290 + y_109918682 + y_111366677 + y_111542178 + y_111549621
 + y_111623049 + y_111676393 + y_111714162 + y_111858325 + y_111862061
 + y_112260708 + y_112260753 + y_112260834 + y_1124840188 + y_1200543263
 + y_1364943411 + y_1365510149 + y_1463828484 + y_1463828485 + y_1463828588
 + y_148576041 + y_148576073 + y_148780506 + y_169008385 + y_169011367
 + y_169018053 + y_169096021 + y_169102608 + y_1838815415 + y_1901353894
 + y_1901353898 + y_200659479 + y_200698653 + y_200902183 + y_200916833
 + y_201011558 + y_201405687 + y_201810722 + y_229405902 + y_27164757
 + y_287901019 + y_287901074 + y_287901081 + y_287913141 + y_310943605
 + y_311343253 + y_

**Formulating the model for a solution**

In [18]:
# Solve the model using PuLP's default solver
model.solve()

# --- Print the results ---
print(f"Status: {LpStatus[model.status]}")

print("\n--- Optimal Solution Found ---")
# Find which intersection was chosen
chosen_intersection_id = None
for i in intersection_ids:
    if location_vars[i].value() == 1:
        chosen_intersection_id = i
        break

if chosen_intersection_id is not None:
    chosen_intersection_info = intersections[intersections['OSM_ID'] == chosen_intersection_id].iloc[0]

    print(f"Optimal Staging Location (OSM_ID): {chosen_intersection_info['OSM_ID']}")
    # Check if 'Name' exists and print it, otherwise print a placeholder
    if 'Name' in chosen_intersection_info and pd.notna(chosen_intersection_info['Name']):
        print(f"Name: {chosen_intersection_info['Name']}")
    else:
        print("Name: Not Available")

    # Check if 'Exit_Number' exists and print it, otherwise print a placeholder
    if 'Exit_Number' in chosen_intersection_info and pd.notna(chosen_intersection_info['Exit_Number']):
         print(f"Exit Number: {chosen_intersection_info['Exit_Number']}")
    else:
        print("Exit Number: Not Available")

    print(f"Minimized Maximum Distance: {max_dist_var.value():.2f} miles")
    print(f"Coordinates: ({chosen_intersection_info['Latitude']}, {chosen_intersection_info['Longitude']})")

else:
    print("No optimal intersection found.")

Status: Optimal

--- Optimal Solution Found ---
Optimal Staging Location (OSM_ID): 425244674
Name: Fort Littleton
Exit Number: 180
Minimized Maximum Distance: 130.96 miles
Coordinates: (40.049731, -77.9607146)


**Visualization for the final optimal location suggested by MIP**

In [20]:
# Create a map to show the solution visually
place_name = "Pennsylvania, USA"
highway_ref = "76" # The reference tag for I-76 in OpenStreetMap

# Using OSMnx to download the network graph for I-76 in Pennsylvania
print(f"Downloading exact geometry for {highway_ref} in {place_name}...")
graph = ox.graph_from_place(place_name, network_type='drive', custom_filter=f'["ref"~"{highway_ref}"]')

# Convert the graph edges (the road segments) into a GeoDataFrame
i76_gdf = ox.graph_to_gdfs(graph, nodes=False, edges=True)

pa_map = folium.Map(location=[40.5, -78.0], zoom_start=7)

#Adding PA map with I-76
i76_geojson = folium.GeoJson(
    i76_gdf,
    style_function=lambda feature: {
        'color': 'orange',
        'weight': 4,
        'opacity': 0.9
    },
    tooltip=f"<b>{highway_ref}</b>"
)

# Add the highway layer to the map
i76_geojson.add_to(pa_map)

C:\Users\vashi\anaconda3\Lib\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 54 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [21]:
# Create a map to show the solution visually the optimal EMS staging location

for idx, row in intersections.iterrows():
    # Check if this intersection is the chosen one to make it green
    try:
        if row['OSM_ID'] == chosen_intersection_info['OSM_ID']:
            icon_color = 'green'
            icon_symbol = 'star'
            popup_text = f"**Optimal Location**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        else:
            icon_color = 'red'
            icon_symbol = 'info-sign'
            popup_text = f"Intersection:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
    except NameError:
        # Error proof
        icon_color = 'red'
        icon_symbol = 'info-sign'
        popup_text = f"Intersection:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"


    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(color=icon_color, icon=icon_symbol)
    ).add_to(pa_map)

# Add markers for crash hotspots, varying size by CRN (count)
max_crn = crash_hotspots['CRN'].max()
min_crn = crash_hotspots['CRN'].min()

for idx, row in crash_hotspots.iterrows():
    popup_text = f"Hotspot:<br>Name: {row.get('28milename', 'N/A')}<br>CRN: {row.get('CRN', 'N/A')}<br>Severity: {row.get('is_severe', 'N/A')}"
    # Scale the radius based on CRN value - so that small circles have lower crash density, larger circles have higher density
    radius = 6 + (row['CRN'] - min_crn) / (max_crn - min_crn) * 24 if max_crn > min_crn else 6


    folium.CircleMarker(
        location=[row['14milelat'], row['14milelon']],
        radius=radius,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(pa_map)


# Display the final map
pa_map

# **Weighted Optimization Model (Weighted by 'CRN' - count/ mile)**

In [23]:
# Calculate weights for hotspots based on CRN
total_crashes = crash_hotspots['CRN'].sum()
crash_hotspots['weight'] = crash_hotspots['CRN'] / total_crashes
hotspot_weights = crash_hotspots.set_index('28milename')['weight'].to_dict()

# 1 - Initializing the Weighted Model
model_weighted = LpProblem("EMS_Location_CrashCountWeighted", LpMinimize)

# 2 - Decision Variables #adding hotspot weights
intersection_ids = intersections['OSM_ID'].tolist()
hotspot_ids = crash_hotspots['28milename'].tolist()
location_vars_weighted = LpVariable.dicts("y_weighted", intersection_ids, cat='Binary')

# 3 - Weighted Optimization Function #minimizing distance(weighted)
model_weighted += lpSum([distance_matrix[i][j] * location_vars_weighted[i] * hotspot_weights[j]
                         for i in intersection_ids for j in hotspot_ids]), "Minimize_Weighted_Total_Distance"

# 4 - Constraint: Choose exactly ONE intersection
model_weighted += lpSum([location_vars_weighted[i] for i in intersection_ids]) == 1, "Choose_Exactly_One_Intersection_Weighted"

model_weighted.solve() #solving

# Print the results for the weighted model
print(f"Status (Weighted Model): {LpStatus[model_weighted.status]}")

print("\n--- Optimal Solution Found (Weighted Model) ---")
# Find which intersection was chosen in the weighted model
chosen_intersection_id_weighted = None
for i in intersection_ids:
    if location_vars_weighted[i].value() == 1:
        chosen_intersection_id_weighted = i
        break

if chosen_intersection_id_weighted is not None:
    chosen_intersection_info_weighted = intersections[intersections['OSM_ID'] == chosen_intersection_id_weighted].iloc[0]
    print(f"Optimal Staging Location (OSM_ID) (Weighted Model): {chosen_intersection_info_weighted['OSM_ID']}")

    if 'Name' in chosen_intersection_info_weighted and pd.notna(chosen_intersection_info_weighted['Name']):
        print(f"Name: {chosen_intersection_info_weighted['Name']}")
    else:
        print("Name: Not Available")

    if 'Exit_Number' in chosen_intersection_info_weighted and pd.notna(chosen_intersection_info_weighted['Exit_Number']):
        print(f"Exit Number: {chosen_intersection_info_weighted['Exit_Number']}")
    else:
        print("Exit Number: Not Available")

    print(f"Minimized Weighted Total Distance: {model_weighted.objective.value():.2f} miles")
    print(f"Coordinates: ({chosen_intersection_info_weighted['Latitude']}, {chosen_intersection_info_weighted['Longitude']})")

else:
    print("No optimal intersection found for the weighted model.")

Status (Weighted Model): Optimal

--- Optimal Solution Found (Weighted Model) ---
Optimal Staging Location (OSM_ID) (Weighted Model): 802260551
Name: Not Available
Exit Number: 236
Minimized Weighted Total Distance: 82.58 miles
Coordinates: (40.1957856, -76.9770611)


**Visualization Comparing Optimal Locations**

In [25]:
# Create a new map for comparison with the same zoom and dimensions for uniformity
comparison_map_weighted_by_count = folium.Map(location=[40.5, -78.0], zoom_start=7)

# Add the I-76 highway layer to the new map
try:
    i76_geojson.add_to(comparison_map_weighted_by_count)
except NameError: # if i76_gdf  unavailable
    print("Regenerating I-76 highway layer...")
    place_name = "Pennsylvania, USA"
    highway_ref = "76" # this is the reference tag for I-76 in OpenStreetMap
    graph = ox.graph_from_place(place_name, network_type='drive', custom_filter=f'["ref"~"{highway_ref}"]')
    i76_gdf = ox.graph_to_gdfs(graph, nodes=False, edges=True)
    i76_geojson = folium.GeoJson(
        i76_gdf,
        style_function=lambda feature: {
            'color': 'orange',
            'weight': 4,
            'opacity': 0.9
        },
        tooltip=f"<b>{highway_ref}</b>"
    )
    i76_geojson.add_to(comparison_map_weighted_by_count)


# Add markers for the optimal intersection locations
for idx, row in intersections.iterrows():
    is_initial_optimal = False
    is_weighted_optimal = False

    # Check if this is the optimal location from the initial model
    try:
        if row['OSM_ID'] == chosen_intersection_info['OSM_ID']:
            is_initial_optimal = True
    except NameError:
        pass # In case initial model results not available #Exception try-except loops

    # Check if this is the optimal location from the weighted model
    try:
        if row['OSM_ID'] == chosen_intersection_info_weighted['OSM_ID']:
            is_weighted_optimal = True
    except NameError:
        pass # Weighted model results not available

    # Only add markers for the optimal locations
    if is_initial_optimal or is_weighted_optimal:
        if is_initial_optimal and is_weighted_optimal:
            # If by chance they are the same location
            icon_color = 'green' # Or a different color to indicate both
            icon_symbol = 'star'
            popup_text = f"/Both Optimal Locations/:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_initial_optimal:
            icon_color = 'green'
            icon_symbol = 'star'
            popup_text = f"/Initial Optimal Location/:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_weighted_optimal:
            icon_color = 'blue' # Different color for weighted optimal
            icon_symbol = 'star'
            popup_text = f"/Weighted Optimal Location/:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            icon=folium.Icon(color=icon_color, icon=icon_symbol)
        ).add_to(comparison_map_weighted_by_count)

# Add markers for crash hotspots, varying size by CRN
max_crn = crash_hotspots['CRN'].max()
min_crn = crash_hotspots['CRN'].min()

for idx, row in crash_hotspots.iterrows():
    popup_text = f"Hotspot:<br>Name: {row.get('28milename', 'N/A')}<br>CRN: {row.get('CRN', 'N/A')}<br>Severity: {row.get('is_severe', 'N/A')}"
    # Scale the radius based on CRN value
    radius = 6 + (row['CRN'] - min_crn) / (max_crn - min_crn) * 24 if max_crn > min_crn else 6


    folium.CircleMarker(
        location=[row['14milelat'], row['14milelon']],
        radius=radius,
        color='purple', # Using a different color for hotspots
        fill=True,
        fill_color='purple',
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(comparison_map_weighted_by_count)


# Display the comparison map
comparison_map_weighted_by_count

# **Weighted Optimization Model (Weighted by 'is_sever')**

In [27]:
# Calculate weights for hotspots based on Severity
total_severe_crashes = crash_hotspots['is_severe'].sum()
crash_hotspots['severity_weight'] = crash_hotspots['is_severe'] / (total_severe_crashes)

# Display the hotspots with their calculated severity weights
display(crash_hotspots[['28milename', 'is_severe', 'severity_weight']].head())

# Store severity weights in a dictionary for the model
hotspot_severity_weights = crash_hotspots.set_index('28milename')['severity_weight'].to_dict()

,28milename,is_severe,severity_weight
0,mile112,1,0.058824
1,mile140,1,0.058824
2,mile168,0,0.000000
3,mile196,1,0.058824
4,mile224,1,0.058824


**Severity Weighted Optimization Model Formulation and Solution**

In [29]:
# 1 - Initializing the Severity Weighted Model
model_severity_weighted = LpProblem("Ambulance_Location_Severity_Weighted", LpMinimize)

# 2 - Decision Variables
intersection_ids = intersections['OSM_ID'].tolist()
hotspot_ids = crash_hotspots['28milename'].tolist()
location_vars_severity_weighted = LpVariable.dicts("y_severity_weighted", intersection_ids, cat='Binary')

# 3 - Severity Weighted Optimization Function #minimize wieghted distance (by
# severity this time)
model_severity_weighted += lpSum([distance_matrix[i][j] * location_vars_severity_weighted[i] * hotspot_severity_weights[j]
                                 for i in intersection_ids for j in hotspot_ids]), "Minimize_Severity_Weighted_Total_Distance"

# 4 - Constraint: Choose exactly ONE intersection
model_severity_weighted += lpSum([location_vars_severity_weighted[i] for i in intersection_ids]) == 1, "Choose_Exactly_One_Intersection_Severity_Weighted"

# Solve
model_severity_weighted.solve()

# Print the results for the severity weighted model
print(f"Status (Severity Weighted Model): {LpStatus[model_severity_weighted.status]}")

print("\n--- Optimal Solution Found (Severity Weighted Model) ---")
# Find which intersection was chosen in the severity weighted model
chosen_intersection_id_severity_weighted = None
for i in intersection_ids:
    if location_vars_severity_weighted[i].value() == 1:
        chosen_intersection_id_severity_weighted = i
        break

if chosen_intersection_id_severity_weighted is not None:
    chosen_intersection_info_severity_weighted = intersections[intersections['OSM_ID'] == chosen_intersection_id_severity_weighted].iloc[0]
    print(f"Optimal Staging Location (OSM_ID) (Severity Weighted Model): {chosen_intersection_info_severity_weighted['OSM_ID']}")

    if 'Name' in chosen_intersection_info_severity_weighted and pd.notna(chosen_intersection_info_severity_weighted['Name']):
        print(f"Name: {chosen_intersection_info_severity_weighted['Name']}")
    else:
        print("Name: Not Available")

    if 'Exit_Number' in chosen_intersection_info_severity_weighted and pd.notna(chosen_intersection_info_severity_weighted['Exit_Number']):
         print(f"Exit Number: {chosen_intersection_info_severity_weighted['Exit_Number']}")
    else:
        print("Exit Number: Not Available")

    print(f"Minimized Severity Weighted Total Distance: {model_severity_weighted.objective.value():.2f} miles")

    print(f"Coordinates: ({chosen_intersection_info_severity_weighted['Latitude']}, {chosen_intersection_info_severity_weighted['Longitude']})")

else:
    print("No optimal intersection found for the severity weighted model.")

Status (Severity Weighted Model): Optimal

--- Optimal Solution Found (Severity Weighted Model) ---
Optimal Staging Location (OSM_ID) (Severity Weighted Model): 802260551
Name: Not Available
Exit Number: 236
Minimized Severity Weighted Total Distance: 78.11 miles
Coordinates: (40.1957856, -76.9770611)


**Visualization Comparing All Three Optimal Locations**

In [31]:
# Create a new map for comparison of all three optimal locations
# Using the same center and zoom as the previous maps
all_optimal_comparison_map = folium.Map(location=[40.5, -78.0], zoom_start=7)

# Add the I-76 highway layer to the new map
try:
    i76_geojson.add_to(all_optimal_comparison_map)
except NameError:
    # If i76_geojson is not available, regenerate the highway layer
    print("Regenerating I-76 highway layer...")
    place_name = "Pennsylvania, USA"
    highway_ref = "76" # The reference tag for I-76 in OpenStreetMap
    graph = ox.graph_from_place(place_name, network_type='drive', custom_filter=f'["ref"~"{highway_ref}"]')
    i76_gdf = ox.graph_to_gdfs(graph, nodes=False, edges=True)
    i76_geojson = folium.GeoJson(
        i76_gdf,
        style_function=lambda feature: {
            'color': 'orange',
            'weight': 4,
            'opacity': 0.9
        },
        tooltip=f"<b>{highway_ref}</b>"
    )
    i76_geojson.add_to(all_optimal_comparison_map)


# Add markers only for the three optimal intersection locations
for idx, row in intersections.iterrows():
    is_initial_optimal = False
    is_weighted_optimal = False
    is_severity_weighted_optimal = False

    # Check if this is the optimal location from the initial model
    try:
        if row['OSM_ID'] == chosen_intersection_info['OSM_ID']:
            is_initial_optimal = True
    except NameError:
        pass # Initial model results not available

    # Check if this is the optimal location from the CRN weighted model
    try:
        if row['OSM_ID'] == chosen_intersection_info_weighted['OSM_ID']:
            is_weighted_optimal = True
    except NameError:
        pass # Weighted model results not available

    # Check if this is the optimal location from the severity weighted model
    try:
        if row['OSM_ID'] == chosen_intersection_info_severity_weighted['OSM_ID']:
            is_severity_weighted_optimal = True
    except NameError:
        pass # Severity weighted model results not available

    # Only add markers for the optimal locations
    if is_initial_optimal or is_weighted_optimal or is_severity_weighted_optimal:
        if is_initial_optimal and is_weighted_optimal and is_severity_weighted_optimal:
             icon_color = 'black' # Color if all three are the same location
             icon_symbol = 'star'
             popup_text = f"**All Three Optimal Locations**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_initial_optimal and is_weighted_optimal:
             icon_color = 'green' # Color if initial and CRN weighted are same
             icon_symbol = 'star'
             popup_text = f"**Initial and CRN Optimal Locations**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_initial_optimal and is_severity_weighted_optimal:
             icon_color = 'red' # Color if initial and severity weighted are same
             icon_symbol = 'star'
             popup_text = f"**Initial and Severity Optimal Locations**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_weighted_optimal and is_severity_weighted_optimal:
             icon_color = 'orange' # Color if CRN weighted and severity weighted are same
             icon_symbol = 'star'
             popup_text = f"**CRN and Severity Optimal Locations**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_initial_optimal:
            icon_color = 'green'
            icon_symbol = 'star'
            popup_text = f"**Initial Optimal Location**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_weighted_optimal:
            icon_color = 'blue' # Different color for CRN weighted optimal
            icon_symbol = 'star'
            popup_text = f"**CRN Weighted Optimal Location**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"
        elif is_severity_weighted_optimal:
            icon_color = 'purple' # Different color for severity weighted optimal
            icon_symbol = 'star'
            popup_text = f"**Severity Weighted Optimal Location**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}<br>OSM ID: {row['OSM_ID']}"

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            icon=folium.Icon(color=icon_color, icon=icon_symbol)
        ).add_to(all_optimal_comparison_map)

# Add markers for crash hotspots, varying size by CRN
max_crn = crash_hotspots['CRN'].max()
min_crn = crash_hotspots['CRN'].min()

for idx, row in crash_hotspots.iterrows():
    popup_text = f"Hotspot:<br>Name: {row.get('28milename', 'N/A')}<br>CRN: {row.get('CRN', 'N/A')}<br>Severity: {row.get('is_severe', 'N/A')}"
    # Scale the radius based on CRN value
    radius = 6 + (row['CRN'] - min_crn) / (max_crn - min_crn) * 24 if max_crn > min_crn else 24


    folium.CircleMarker(
        location=[row['14milelat'], row['14milelon']],
        radius=radius,
        color='purple', # Using a neutral color for hotspots on this map
        fill=True,
        fill_color='purple',
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(all_optimal_comparison_map)


# Display the comparison map
all_optimal_comparison_map

In [33]:
import numpy as np
from scipy.spatial import cKDTree

# Prepare coordinates from both dataframes
hotspot_coords = crash_hotspots[['14milelat', '14milelon']].values
intersection_coords = intersections[['Latitude', 'Longitude']].values

# Build a k-d tree for fast nearest-neighbor lookup
hotspot_tree = cKDTree(hotspot_coords)

# Find the index of the nearest hotspot for each intersection
# The query returns distance and index; we only need the index
distances, indices = hotspot_tree.query(intersection_coords, k=1)

# Add a new column to the intersections dataframe with the assigned hotspot
intersections['assigned_hotspot'] = crash_hotspots.loc[indices, '28milename'].values

# Create a dictionary to group intersection OSM_IDs by their assigned hotspot
stretch_groups = intersections.groupby('assigned_hotspot')['OSM_ID'].apply(list).to_dict()

# Print a sample group to see the result
print("Exits assigned to 'mile344':")
print(stretch_groups.get('mile344', 'No exits found for this stretch'))

Exits assigned to 'mile344':
No exits found for this stretch


# Multi-Vehicle Ambulance Staging Model

This model is designed to select 13 optimal ambulance staging locations, with the constraint that exactly one location is chosen from each of the 13 predefined highway stretches. The objective is to minimize the total response distance, weighted by the severity of crashes at each hotspot.

## Sets
* $I$: The set of all potential ambulance staging locations (exits).
* $J$: The set of all crash hotspots (the 13 stretches).
* $K$: The set of 13 predefined stretches.
* $I_k$: The subset of locations from $I$ that geographically belong to stretch $k \in K$.

***

## Parameters
* $d_{ij}$: The calculated travel distance between a potential location $i \in I$ and a crash hotspot $j \in J$.
* $s_j$: The severity weight of a crash hotspot $j \in J$, calculated from the `is_severe` data.

***

## Decision Variables
* $y_i \in \{0, 1\}$: A binary variable that equals **1** if an ambulance is staged at location $i$, and **0** otherwise.
* $x_{ij} \in \{0, 1\}$: A binary variable that equals **1** if crash hotspot $j$ is assigned to be covered by the ambulance at location $i$, and **0** otherwise.

***

## Objective Function

The goal is to minimize the sum of all severity-weighted travel distances. This is achieved by multiplying the distance from an assigned ambulance to a hotspot by that hotspot's severity weight.

$$\min \sum_{i \in I} \sum_{j \in J} s_j \cdot d_{ij} \cdot x_{ij}$$

***

## Constraints

The model must adhere to the following rules:

$$
\begin{align*}
\sum_{i \in I} x_{ij} &= 1 && \forall j \in J && \text{(1. Each hotspot must be assigned to one ambulance.)} \\
x_{ij} &\le y_i && \forall i \in I, \forall j \in J && \text{(2. A hotspot can only be assigned to an active ambulance location.)} \\
\sum_{i \in I_k} y_i &= 1 && \forall k \in K && \text{(3. Exactly one ambulance must be placed in each stretch.)}
\end{align*}
$$

**Multi-Vehicle Ambulance Staging Model Summary**

*   **Parameters:** $d_{ij}$ (distance from location $i$ to hotspot $j$), $s_j$ (severity weight of hotspot $j$).
*   **Decision Variables:** $y_i$ (binary for placing ambulance at $i$), $x_{ij}$ (binary for assigning hotspot $j$ to location $i$).
*   **Objective:** Minimize $\sum_{i,j} s_j \cdot d_{ij} \cdot x_{ij}$ (Minimize total severity-weighted distance).
*   **Constraints:** 1. Each hotspot assigned to one ambulance: $\sum_i x_{ij} = 1$ for all $j$.
*   2. Assignment requires ambulance: $x_{ij} \le y_i$ for all $i, j$.
*   3. One ambulance per stretch: $\sum_{i \in I_k} y_i = 1$ for each stretch $k$.

In [36]:
# 1 - Initializing the final multi-vehicle model
model_final = LpProblem("Ambulance_Staging_13_Vehicles", LpMinimize)

# Get lists of IDs
intersection_ids = intersections['OSM_ID'].tolist()
hotspot_ids = crash_hotspots['28milename'].tolist()
stretch_ids = list(stretch_groups.keys())

# 2 - Decision Variables
# y_i: 1 if we place an ambulance at intersection i
y = LpVariable.dicts("y_location", intersection_ids, cat='Binary')

# x_ij: 1 if hotspot j is assigned to an ambulance at intersection i
x = LpVariable.dicts("x_assignment", (intersection_ids, hotspot_ids), cat='Binary')

# 3 - Objective Function
# Minimize the total severity-weighted distance
model_final += lpSum([hotspot_severity_weights[j] * distance_matrix[i][j] * x[i][j]
                      for i in intersection_ids for j in hotspot_ids]), "Minimize_Total_Severity_Weighted_Distance"

# 4 - Constraints
# Constraint 1: Each hotspot must be assigned to exactly one location
for j in hotspot_ids:
    model_final += lpSum([x[i][j] for i in intersection_ids]) == 1, f"Assign_Hotspot_{j}"

# Constraint 2: A hotspot can only be assigned to an open location
for i in intersection_ids:
    for j in hotspot_ids:
        model_final += x[i][j] <= y[i], f"Activation_{i}_{j}"

# Constraint 3: Exactly one ambulance must be placed per stretch
for k in stretch_ids:
    # stretch_groups[k] contains the list of intersection_ids for that stretch
    model_final += lpSum([y[i] for i in stretch_groups[k]]) == 1, f"One_Ambulance_per_Stretch_{k}"


# --- Solve and Print Results ---
model_final.solve()

print(f"Status: {LpStatus[model_final.status]}")
print(f"Total Minimized Weighted Distance: {model_final.objective.value():.2f} miles\n")

print("--- Optimal Ambulance Locations ---")
chosen_locations = []
for i in intersection_ids:
    if y[i].value() == 1:
        chosen_locations.append(i)
        loc_info = intersections[intersections['OSM_ID'] == i].iloc[0]
        print(f"Stretch '{loc_info['assigned_hotspot']}': Place ambulance at Exit {loc_info['Exit_Number']}, {loc_info.get('Name', 'N/A')} (OSM ID: {i})")

# Wehave a list called 'chosen_locations' with the OSM_IDs of the 13 optimal spots.

Status: Optimal
Total Minimized Weighted Distance: 4.04 miles

--- Optimal Ambulance Locations ---
Stretch 'mile56': Place ambulance at Exit 39, Butler Valley (OSM ID: 713282313)
Stretch 'mile84': Place ambulance at Exit 67, Irwin (OSM ID: 1365510149)
Stretch 'mile112': Place ambulance at Exit 91, Donegal (OSM ID: 444312134)
Stretch 'mile168': Place ambulance at Exit 146, Bedford (OSM ID: 444620246)
Stretch 'mile196': Place ambulance at Exit 180, Fort Littleton (OSM ID: 32907604)
Stretch 'mile252': Place ambulance at Exit 236, nan (OSM ID: 802260551)
Stretch 'mile280': Place ambulance at Exit 266, Lebanon-Lancaster (OSM ID: 6067520486)
Stretch 'mile308': Place ambulance at Exit 298, Morgantown (OSM ID: 229405902)
Stretch 'mile336': Place ambulance at Exit 320, SR 29 (OSM ID: 1046413828)
Stretch 'mile224': Place ambulance at Exit nan, nan (OSM ID: 111858325)
Stretch 'mile28': Place ambulance at Exit nan, BEAVER VALLEY (OSM ID: 746197933)


In [37]:
# Create a new map to visualize the final 13 locations
final_map = folium.Map(location=[40.5, -78.0], zoom_start=7)

# Add the I-76 highway layer
i76_geojson.add_to(final_map)

# Add markers for all potential intersections, highlighting the 13 chosen optimal locations
for idx, row in intersections.iterrows():
    if row['OSM_ID'] in chosen_locations:
        icon_color = 'green'
        icon_symbol = 'star'
        popup_text = f"**Optimal Location for Stretch '{row['assigned_hotspot']}'**:<br>Name: {row.get('Name', 'N/A')}<br>Exit: {row.get('Exit_Number', 'N/A')}"
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            icon=folium.Icon(color=icon_color, icon=icon_symbol)
        ).add_to(final_map)

# Add markers for crash hotspots, now sized by SEVERITY
max_severe = crash_hotspots['is_severe'].max()
min_severe = crash_hotspots['is_severe'].min()

for idx, row in crash_hotspots.iterrows():
    popup_text = f"Hotspot:<br>Name: {row.get('28milename', 'N/A')}<br>CRN: {row.get('CRN', 'N/A')}<br>Severity: {row.get('is_severe', 'N/A')}"

    # Scale the radius based on the 'is_severe' value.
    radius = 6 + (row['is_severe'] - min_severe) / (max_severe - min_severe) * 24 if max_severe > min_severe else 6

    folium.CircleMarker(
        location=[row['14milelat'], row['14milelon']],
        radius=radius,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(final_map)

# Save and display the final map
final_map.save("final_severity_locations_map.html")
final_map

# Experimental table for future work - dashboarding the "current" optimal locations, and their proximity to segment centres - for agency and administrative consumption

## Can add several dependencies in the model, like seasonality, time of day weights, directional weights, vehicular categories for crash vehicle (as well as EMS) and resulting dependencies, traffic conditions, people per crash, exit locational proximity for EMS etc, for furthering robustness and granularity for real-world use.

For the following section, this is an initial effort to tabulate for every hotspot its own severity, versus the severity of its preceding and following hotspots, and calculate the minimum time it would take for the EMS within that hotspot to travel to the farthest point within its own segment, and also the farthest point of its neighboring segments.

In [40]:
# 1. Create an empty dictionary to store the results
max_distance_within_segment = {}

# 2. Iterate through each chosen location
for chosen_location_id in chosen_locations:
    # 3. Find the corresponding row in the intersections DataFrame to get its assigned_hotspot
    location_info = intersections[intersections['OSM_ID'] == chosen_location_id].iloc[0]
    assigned_hotspot = location_info['assigned_hotspot']

    # 4. Filter the crash_hotspots DataFrame to get only the hotspots that belong to the current assigned_hotspot segment
    # In this case, each hotspot represents a segment, so we just need the distance to the assigned hotspot
    # We assume that the "segment" is defined by the assigned hotspot location itself
    hotspot_in_segment_id = assigned_hotspot

    # 5. Find the distance from the current chosen location to the hotspot within its segment
    # Access the distance from the distance_matrix
    distance_to_assigned_hotspot = distance_matrix[chosen_location_id][hotspot_in_segment_id]

    # 6. Determine the maximum distance among these distances.
    # Since we are considering the distance to the single assigned hotspot in this interpretation,
    # the maximum distance within the segment is simply this distance.
    max_dist = distance_to_assigned_hotspot

    # 7. Store this maximum distance in the max_distance_within_segment dictionary
    max_distance_within_segment[assigned_hotspot] = max_dist

# Print the results
print("Maximum distance from each chosen location to its assigned hotspot (within its segment):")
for segment, max_dist in max_distance_within_segment.items():
    print(f"Segment '{segment}': {max_dist:.2f} miles")

Maximum distance from each chosen location to its assigned hotspot (within its segment):
Segment 'mile56': 3.53 miles
Segment 'mile84': 3.14 miles
Segment 'mile112': 7.38 miles
Segment 'mile168': 7.82 miles
Segment 'mile196': 1.83 miles
Segment 'mile252': 1.55 miles
Segment 'mile280': 0.50 miles
Segment 'mile308': 4.30 miles
Segment 'mile336': 1.68 miles
Segment 'mile224': 6.86 miles
Segment 'mile28': 8.06 miles


In [41]:
# Get the list of unique hotspot names from the crash_hotspots DataFrame.
hotspot_names = crash_hotspots['28milename'].unique().tolist()

# Create a dictionary to store preceding and following hotspots.
hotspot_neighbors = {}

# Iterate through the list of hotspot names to find neighbors.
for i, hotspot_name in enumerate(hotspot_names):
    preceding_hotspot = hotspot_names[i - 1] if i > 0 else None
    following_hotspot = hotspot_names[i + 1] if i < len(hotspot_names) - 1 else None

    hotspot_neighbors[hotspot_name] = {
        'preceding': preceding_hotspot,
        'following': following_hotspot
    }

# Print the result
print("Hotspot Neighbors (preceding and following segments):")
for hotspot, neighbors in hotspot_neighbors.items():
    print(f"Hotspot '{hotspot}': Preceding: {neighbors['preceding']}, Following: {neighbors['following']}")

Hotspot Neighbors (preceding and following segments):
Hotspot 'mile112': Preceding: None, Following: mile140
Hotspot 'mile140': Preceding: mile112, Following: mile168
Hotspot 'mile168': Preceding: mile140, Following: mile196
Hotspot 'mile196': Preceding: mile168, Following: mile224
Hotspot 'mile224': Preceding: mile196, Following: mile252
Hotspot 'mile252': Preceding: mile224, Following: mile28
Hotspot 'mile28': Preceding: mile252, Following: mile280
Hotspot 'mile280': Preceding: mile28, Following: mile308
Hotspot 'mile308': Preceding: mile280, Following: mile336
Hotspot 'mile336': Preceding: mile308, Following: mile56
Hotspot 'mile56': Preceding: mile336, Following: mile84
Hotspot 'mile84': Preceding: mile56, Following: None


In [44]:
# 1. Create an empty dictionary to store the maximum distances to neighboring segments
max_distance_to_neighboring_segments = {}

# 2. Iterate through each chosen ambulance location
for chosen_location_id in chosen_locations:
    # 3. Find its corresponding row in the intersections DataFrame to get its assigned_hotspot
    location_info = intersections[intersections['OSM_ID'] == chosen_location_id].iloc[0]
    assigned_hotspot = location_info['assigned_hotspot']

    # 4. Use the hotspot_neighbors dictionary to find the preceding and following hotspots
    neighbors = hotspot_neighbors.get(assigned_hotspot, {})
    preceding_hotspot = neighbors.get('preceding')
    following_hotspot = neighbors.get('following')

    # 5. Initialize a variable to keep track of the maximum distance to neighboring segments
    max_dist_neighbor = 0.0

    # 6. If a preceding hotspot exists, get the distance and update max_dist_neighbor
    if preceding_hotspot:
        dist_to_preceding = distance_matrix[chosen_location_id][preceding_hotspot]
        max_dist_neighbor = max(max_dist_neighbor, dist_to_preceding)

    # 7. If a following hotspot exists, get the distance and update max_dist_neighbor
    if following_hotspot:
        dist_to_following = distance_matrix[chosen_location_id][following_hotspot]
        max_dist_neighbor = max(max_dist_neighbor, dist_to_following)

    # 8. Store the maximum distance to neighboring segments in the dictionary
    max_distance_to_neighboring_segments[assigned_hotspot] = max_dist_neighbor

# 9. Print the maximum distances to neighboring segments for each chosen location's assigned hotspot
print("Maximum distance from each chosen location to its neighboring segments:")
for segment, max_dist in max_distance_to_neighboring_segments.items():
    print(f"Segment '{segment}': {max_dist:.2f} miles")

Maximum distance from each chosen location to its neighboring segments:
Segment 'mile56': 237.38 miles
Segment 'mile84': 22.18 miles
Segment 'mile112': 31.92 miles
Segment 'mile168': 31.28 miles
Segment 'mile196': 27.65 miles
Segment 'mile252': 178.06 miles
Segment 'mile280': 205.35 miles
Segment 'mile308': 30.36 miles
Segment 'mile336': 232.36 miles
Segment 'mile224': 33.97 miles
Segment 'mile28': 207.35 miles


In [45]:
# 1. Create a dictionary to store the severity of each hotspot.
hotspot_severity = {}

# 2. Iterate through the crash_hotspots DataFrame.
# 3. For each hotspot, store its 'is_severe' value in the dictionary with the hotspot name as the key.
for index, row in crash_hotspots.iterrows():
    hotspot_name = row['28milename']
    severity = row['is_severe']
    hotspot_severity[hotspot_name] = severity

# 4. Create another dictionary to store the severity of the neighboring hotspots.
neighboring_severity = {}

# 5. Iterate through the hotspot_neighbors dictionary.
for hotspot_name, neighbors in hotspot_neighbors.items():
    preceding_hotspot = neighbors.get('preceding')
    following_hotspot = neighbors.get('following')

    # 6. For each hotspot, look up its preceding and following neighbors.
    # 7. If a neighbor exists, retrieve its severity from the hotspot severity dictionary.
    preceding_severity = hotspot_severity.get(preceding_hotspot) if preceding_hotspot else None
    following_severity = hotspot_severity.get(following_hotspot) if following_hotspot else None

    # 8. Store the severity of the preceding and following neighbors in the neighboring severity dictionary, using the original hotspot name as the key.
    neighboring_severity[hotspot_name] = {
        'preceding_severity': preceding_severity,
        'following_severity': following_severity
    }

# 9. Print the dictionaries containing the severity of each hotspot and its neighbors.
print("Severity of Each Hotspot:")
print(hotspot_severity)

print("\nSeverity of Neighboring Hotspots:")
print(neighboring_severity)

Severity of Each Hotspot:
{'mile112': 1, 'mile140': 1, 'mile168': 0, 'mile196': 1, 'mile224': 1, 'mile252': 2, 'mile28': 1, 'mile280': 2, 'mile308': 2, 'mile336': 3, 'mile56': 2, 'mile84': 1}

Severity of Neighboring Hotspots:
{'mile112': {'preceding_severity': None, 'following_severity': 1}, 'mile140': {'preceding_severity': 1, 'following_severity': 0}, 'mile168': {'preceding_severity': 1, 'following_severity': 1}, 'mile196': {'preceding_severity': 0, 'following_severity': 1}, 'mile224': {'preceding_severity': 1, 'following_severity': 2}, 'mile252': {'preceding_severity': 1, 'following_severity': 1}, 'mile28': {'preceding_severity': 2, 'following_severity': 2}, 'mile280': {'preceding_severity': 1, 'following_severity': 2}, 'mile308': {'preceding_severity': 2, 'following_severity': 3}, 'mile336': {'preceding_severity': 2, 'following_severity': 2}, 'mile56': {'preceding_severity': 3, 'following_severity': 1}, 'mile84': {'preceding_severity': 2, 'following_severity': None}}


In [47]:
# --- Part 1: Extract the assignments from the solved optimization model ---
# The model's 'x' variables tell us which hotspot is assigned to which location.
# We'll add this information to our 'intersections' DataFrame.

# Ensure the new column exists
intersections['assigned_hotspot'] = None

# For each hotspot, find which of the chosen locations is assigned to it
for j in hotspot_ids:
    for i in chosen_locations:
        # Check the value of the x[i][j].value() decision variable
        if x[i][j].value() == 1:
            # If the value is 1, this assignment was made.
            # Store the name of the hotspot 'j' in the row for location 'i'.
            intersections.loc[intersections['OSM_ID'] == i, 'assigned_hotspot'] = j
            break # Assignment found, move to the next hotspot

# --- Part 2: Build the skew analysis table ---
# Now we'll use our updated DataFrame to create the final table.

skew_analysis_data = []

# Filter the intersections DataFrame to only our 13 chosen locations
chosen_locations_df = intersections[intersections['OSM_ID'].isin(chosen_locations)].copy()

# Iterate through each of the 13 optimal ambulance locations
for index, row in chosen_locations_df.iterrows():
    segment_k = row['assigned_hotspot']
    optimal_location_k_id = row['OSM_ID']

    # Get the names of the neighboring segments from our previously created dictionary
    neighbors = hotspot_neighbors.get(segment_k, {})
    segment_k_minus_1 = neighbors.get('preceding')
    segment_k_plus_1 = neighbors.get('following')

    # Initialize variables for the table
    severity_k_minus_1, dist_to_k_minus_1 = None, None
    severity_k_plus_1, dist_to_k_plus_1 = None, None

    # Get data for the preceding segment (k-1) if it exists
    if segment_k_minus_1:
        severity_k_minus_1 = hotspot_severity.get(segment_k_minus_1)
        dist_to_k_minus_1 = distance_matrix[optimal_location_k_id][segment_k_minus_1]

    # Get data for the following segment (k+1) if it exists
    if segment_k_plus_1:
        severity_k_plus_1 = hotspot_severity.get(segment_k_plus_1)
        dist_to_k_plus_1 = distance_matrix[optimal_location_k_id][segment_k_plus_1]

    # Append all the compiled information for the current segment
    skew_analysis_data.append({
        'Segment (k)': segment_k,
        'Preceding (k-1)': segment_k_minus_1,
        'Severity (k-1)': severity_k_minus_1,
        'Dist from k to k-1 (miles)': dist_to_k_minus_1,
        'Dist from k to k+1 (miles)': dist_to_k_plus_1,
        'Severity (k+1)': severity_k_plus_1,
        'Following (k+1)': segment_k_plus_1
    })

# --- Part 3: Display the final table ---

# Create a pandas DataFrame from the collected data
summary_table = pd.DataFrame(skew_analysis_data)

# Reorder columns for better readability
summary_table = summary_table[[
    'Segment (k)',
    'Severity (k-1)',
    'Dist from k to k-1 (miles)',
    'Dist from k to k+1 (miles)',
    'Severity (k+1)'
]]

# Define a custom sorting order based on the numerical part of the segment name
def sort_key(segment_name):
    try:
        # Extract the number from the segment name (e.g., 'mile112' -> 112)
        return int(segment_name.replace('mile', ''))
    except ValueError:
        # Handle cases that don't match the 'mileXX' format, if any
        return float('inf') # Put them at the end

# Apply the custom sorting order
summary_table['sort_key'] = summary_table['Segment (k)'].apply(sort_key)
summary_table = summary_table.sort_values(by='sort_key').drop('sort_key', axis=1)


# Display the final, insightful table
print("Analysis of Placement Skew Towards Neighboring Severity")
display(summary_table.round(2)) #Not fully correct. #We have understood the
# error to be originating from the straight line distance calculation. Rerun
# with new calculations using OSMNx route distance

Analysis of Placement Skew Towards Neighboring Severity


,Segment (k),Severity (k-1),Dist from k to k-1 (miles),Dist from k to k+1 (miles),Severity (k+1)
10,mile28,2.0,182.25,207.35,2.0
0,mile56,3.0,237.38,28.05,1.0
1,mile84,2.0,22.18,NaN,NaN
2,mile112,NaN,NaN,31.92,1.0
3,mile140,1.0,38.53,7.82,0.0
4,mile196,0.0,22.76,27.65,1.0
9,mile224,1.0,19.05,33.97,2.0
5,mile252,1.0,25.67,178.06,1.0
6,mile280,1.0,205.35,25.60,2.0
7,mile308,2.0,30.36,21.42,3.0
